# 運動のシミュレーション

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

ニュートン力学によると、質点の運動は

<!-- textlint-enable -->

$$
m \frac{d^2 \vec{r}}{dt^2} = \vec{F} ,
\tag{1}
$$

<!-- textlint-disable ja-technical-writing/sentence-length -->

のような常微分方程式（ordinary differential equation）に支配されている<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)。ここで$m$は質点の質量、$\vec{r}$は質点の位置、$\vec{F}$は質点に働く力、$t$は時間である。もし力$\vec{F}$が既知であり、初期条件として時刻$t = t_0$での質点の位置と速度が与えられたならば、その後の質点の運動は常微分方程式<!-- eqref -->(1)の初期値問題（initial value problem）を解くことによって完全に決定される。

<!-- textlint-enable -->

今回は常微分方程式の初期値問題の簡単な数値解法を学び、質点の運動をシミュレーションしてみよう。

<!-- textlint-disable ja-technical-writing/max-ten,ja-technical-writing/no-doubled-joshi -->

**注意**：実際の研究では、よく整備されたライブラリが使えるのなら、それを利用すべきである。今回の場合で言うと、たとえば[SciPy](https://scipy.org/)というライブラリに[`scipy.integrate.solve_ivp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)という関数がある。また、今回主に扱うオイラー法は、数学的にもプログラム的にも簡単で教育的ではあるが、誤差も比較的大きく、理論的な安定性も芳しくないので、常微分方程式の数値解法としてはあまり実用的であるとは言えない。

<!-- textlint-enable -->

## 今回のねらい

- オイラー法を用いた常微分方程式の初期値問題の解法について理解する。
- 数値計算アルゴリズムによる結果には誤差があり、（使い方を間違えると）大きく間違った数値解が得られることもあると実感する。

## オイラー法

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

ここでは少し一般的に、ある$n$個の物理量の組をベクトル$\vec{X} = \left(X_1,\dots,X_n\right)$のように書いて、その時間変化を考える<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2)<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3)。$\vec{X}$の時間についての1階微分が、既知の関数$\vec{f}(\vec{X},t)$によって

<!-- textlint-enable -->

$$
\frac{d \vec{X}(t)}{dt} = \vec{f}(\vec{X}(t),t) ,
\tag{2}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

のように与えられていると仮定しよう。十分に小さな時間変化$\Delta t$に対し、$\vec{X}(t+\Delta t)$のテイラー展開を考えると、

<!-- textlint-enable -->

$$
\begin{split}
\vec{X}(t + \Delta t)
&= \vec{X}(t) + \frac{d \vec{X}(t)}{dt} \Delta t + \mathcal{O} \left( (\Delta t)^2 \right) \\ 
&= \vec{X}(t) + \vec{f}(\vec{X}(t),t) \Delta t + \mathcal{O} \left( (\Delta t)^2 \right) ,
\end{split}
\tag{3}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/sentence-length -->

となる。式<!-- eqref -->(3)の右辺の剰余項$\mathcal{O}\left((\Delta t)^2\right)$を無視すれば、時刻$t$での$\vec{X}$と$\vec{f}$の値から、時刻$(t + \Delta t)$での$\vec{X}$の値を近似的に求めることができる：

<!-- textlint-enable -->

$$
\vec{X}(t + \Delta t)
\simeq
\vec{X}(t) + \vec{f}(\vec{X}(t),t) \Delta t .
\tag{4}
$$

よって、ある初期時刻$t = t_0$において$\vec{X}(t_0)$の値が与えられていれば、式<!-- eqref -->(4)を繰り返し適用することで、その後の$\vec{X}$の時間発展が分かる。式<!-- eqref -->(4)においてステップ幅$\Delta t$を小さくすれば近似がよくなるが<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)、当然その分だけ計算に時間がかかることになる。式<!-- eqref -->(4)を用いた常微分方程式の初期値問題の近似解法をオイラー法（Euler method）と呼ぶ<a name="cite_ref-5"></a>[<sup>[5]</sup>](#cite_note-5)。

<!-- textlint-disable ja-technical-writing/max-kanji-continuous-len -->

## 1次元調和振動子

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/max-kanji-continuous-len -->

ばね振り子のような、1次元調和振動子（one-dimensional harmonic oscillator）の運動の様子を、オイラー法を用いて数値的に計算してみよう。解くべき微分方程式は

<!-- textlint-enable -->

$$
\frac{d^2 x(t)}{dt^2} = - \omega^2 x(t) ,
\tag{5}
$$

である。ここで$x$は調和振動子の変位、$\omega$は角振動数である<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/max-kanji-continuous-len -->

式<!-- eqref -->(5)は2階の微分方程式であるが、速度を表す変数として$v_x = \frac{dx}{dt}$を導入すると、連立の1階微分方程式として書き直すことができる：

<!-- textlint-enable -->

$$
\begin{align}
\frac{dx(t)}{dt} &= v_x(t) \tag{6}, \\
\frac{dv_x(t)}{dt} &= - \omega^2 x(t) \tag{7} .
\end{align}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

よって、$\vec{X} = (x, v_x)$として式<!-- eqref -->(4)を適用すると、

<!-- textlint-enable -->

$$
\begin{align}
x(t+\Delta t) &\simeq x(t) + v_x(t) \Delta t ,
\tag{8} \\
v_x(t+\Delta t) &\simeq v_x(t) - \omega^2 x(t) \Delta t ,
\tag{9}
\end{align}
$$

となる。時刻$t = 0$での初期条件$x(0)$と$v_x(0)$が与えられれば、式<!-- eqref -->(8)と<!-- eqref -->(9)のステップを繰り返すことにより、系の時間発展を近似的に求めることができる。オイラー法による調和振動子のシミュレーションのプログラムを以下に示す<a name="cite_ref-7"></a>[<sup>[7]</sup>](#cite_note-7)。

In [ ]:
# オイラー法による調和振動子のシミュレーション

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# 最終時刻
t_max = 2

# ステップ幅
dt = 0.00001

# 初期条件と角振動数
x0 = 1
vx0 = 0
omega = 4 * np.pi

# 初期化
t = 0
x = x0
vx = vx0

# グラフ用のリスト
t_points = [t]
x_points = [x]
vx_points = [vx]

while t < t_max:
    # オイラー法で時間をdt進める
    x_new = x + vx * dt
    vx_new = vx - omega**2 * x * dt

    t = t + dt
    x = x_new
    vx = vx_new

    # グラフ用に点を追加
    t_points.append(t)
    x_points.append(x)
    vx_points.append(vx)

# グラフを描く
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)

ax1.plot(t_points, x_points, label="Euler")
ax2.plot(t_points, vx_points, label="Euler")

ax1.set_ylabel("$x$")
ax1.grid()
ax1.legend()

ax2.set_xlabel("$t$")
ax2.set_ylabel("$v_x$")
ax2.grid()
ax2.legend()

plt.show()

ここでは[subplots関数](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html)にオプション引数を渡して、横軸を共有した[Axesオブジェクト](https://matplotlib.org/stable/api/axes_api.html#the-axes-class)を2つ（`ax1`と`ax2`）作り、それぞれに$x$と$v_x$の時間変化を描いている。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-ten -->

単位系はどのようにとってもよいが、時間の単位を秒、長さの単位をcmにとって、ばねで天井から吊るされた重りを（重力とばねの復元力が釣り合っている）自然長からいくらか伸ばしたところで静かに手を離した状況を考えると、この結果の物理的な意味をイメージしやすいだろう。

<!-- textlint-enable -->

## 練習問題

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

(1) この調和振動子の運動の解析解は

<!-- textlint-enable -->

$$
\begin{align}
x(t) &= x(0) \cos(\omega t) + \frac{v_x(0)}{\omega} \sin(\omega t) , \tag{10} \\
v_x(t) &= v_x(0) \cos(\omega t) -x(0) \, \omega \sin(\omega t) , \tag{11} \\
\end{align}
$$

で与えられる。これらの曲線をグラフに（たとえば点線で）描き入れ、数値解と比較せよ。

(2) 初期条件$x(0)$、$v_x(0)$や角振動数$\omega$を少し変えてみて、運動がどのように変化するか観察せよ。

(3) ステップ幅$\Delta t$を大きくしてみよ。たとえば、$\Delta t = 0.01$として、どのような結果が得られるか確認せよ<a name="cite_ref-8"></a>[<sup>[8]</sup>](#cite_note-8)。また、そのような結果が得られた理由を考察せよ。

## 斜方投射

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

ボールを斜め上の方向に投げるときのことを考えよう。ボールは質量$m$の質点であるとする（つまり回転などは考えない<a name="cite_ref-9"></a>[<sup>[9]</sup>](#cite_note-9)）。水平方向に$x$軸、鉛直上向きに$y$軸をとる。速度の2乗に比例した空気抵抗（$- b |\vec{v}|^2 (\vec{v} / |\vec{v}|)$）を仮定すると<a name="cite_ref-10"></a>[<sup>[10]</sup>](#cite_note-10)、解くべき微分方程式は

<!-- textlint-enable -->

$$
\begin{align}
\frac{d x(t)}{dt} &= v_x(t) , \tag{12} \\
\frac{d y(t)}{dt} &= v_y(t) , \tag{13} \\
\frac{d v_x(t)}{dt} &= - \frac{b}{m} v(t) v_x(t) , \tag{14} \\
\frac{d v_y(t)}{dt} &= - \frac{b}{m} v(t) v_y(t) - g, \tag{15} \\
\end{align}
$$

である。ここで、$b$は空気抵抗係数、$g$は重力加速度である。また、$v(t)$は速度の大きさ（絶対値）を表し、$v(t) = \sqrt{[v_x(t)]^2 + [v_y(t)]^2}$のように与えられる。時刻$t = 0$に初速度$v_0$で水平方向から角度$\theta$だけ上の向きにボールを投げたとしよう。MKS単位系で適当なパラメーターの値<a name="cite_ref-11"></a>[<sup>[11]</sup>](#cite_note-11)をとったシミュレーションのプログラムは次のようになる<a name="cite_ref-12"></a>[<sup>[12]</sup>](#cite_note-12)。

In [ ]:
# オイラー法による斜方投射のシミュレーション

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# ステップ幅
dt = 0.00001

# パラメーター
m = 0.2
g = 9.8
b = 0.002
v0 = 20
y0 = 1.7  # ボールを離す高さ（人間の身長程度）
theta = np.radians(45)  # （水平方向から測った）投射角

# 初期化
t = 0
x = 0
y = y0
vx = v0 * np.cos(theta)
vy = v0 * np.sin(theta)

# グラフ用のリスト
x_points = [x]
y_points = [y]

# 地面に接触するまでループする
while y >= 0:
    # オイラー法で時間をdt進める
    v = np.sqrt(vx**2 + vy**2)

    x_new = x + vx * dt
    y_new = y + vy * dt
    vx_new = vx - b / m * v * vx * dt
    vy_new = vy - (b / m * v * vy + g) * dt

    t = t + dt
    x = x_new
    y = y_new
    vx = vx_new
    vy = vy_new

    # グラフ用に点を追加
    x_points.append(x)
    y_points.append(y)

# 接地した時の時刻と座標を表示
print(f"t = {t}, x = {x}, y = {y}")

# グラフを描く
fig, ax = plt.subplots()
ax.plot(x_points, y_points)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
ax.set_aspect("equal")
ax.grid()
plt.show()

## 練習問題

(4) 空気抵抗がなく（$b=0$ kg/m）地面の高さ（$y(0)=0$ m）からボールを投げた場合、同じ初速度でもっともボールを遠くまで投げることができる角度は45°である。さて、このシミュレーションのように空気抵抗があり、ボールを投げる高さもある場合、もっともボールを遠くまで投げることのできる角度はどうなるだろうか。次の中から選べ。

(a) 45°より小さな角度。

(b) 45°のまま変わらない。

(c) 45°より大きな角度。

(5) 月面（$g = 1.6$ m/s$^2$、$b = 0$ kg/m）でボール投げをすると、地球の場合と同じ初速度、同じ角度でどのくらい遠くまでボールを投げることができるだろうか。シミュレーションで確かめてみよう。

## 演習課題

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/no-doubled-joshi -->

太陽の周りを動く1つの天体（惑星や彗星など）の運動を考える。天体の質量は太陽質量に比べて無視できるとする。太陽も天体も質点として議論することにすると<a name="cite_ref-13"></a>[<sup>[13]</sup>](#cite_note-13)、ニュートンの万有引力の法則（逆2乗の法則）により運動方程式は次のようになる：

<!-- textlint-enable -->

$$
m \frac{d^2 \vec{r}}{dt^2} = - \frac{GMm}{|\vec{r}|^2} \frac{\vec{r}}{|\vec{r}|} .
\tag{16}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-technical-writing/max-kanji-continuous-len -->

ここで$\vec{r}$は太陽を原点としたときの天体の位置、$t$は時間、$M$と$m$はそれぞれ太陽と天体の質量、$G$は万有引力定数である。中心力のみが働いているので、この運動は平面運動となる。その平面内に2次元直交座標系（$x$軸と$y$軸）をとると、式<!-- eqref -->(16)は次のような連立微分方程式として書き下せる：

<!-- textlint-enable -->

$$
\begin{align}
\frac{d x(t)}{dt} &= v_x(t) , \tag{17} \\
\frac{d y(t)}{dt} &= v_y(t) , \tag{18} \\
\frac{d v_x(t)}{dt} &= - \frac{GM}{[r(t)]^3} x(t) , \tag{19} \\
\frac{d v_y(t)}{dt} &= - \frac{GM}{[r(t)]^3} y(t) . \tag{20} \\
\end{align}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-engineering-paper/prh -->

ただし$r(t) = \sqrt{[x(t)]^2 + [y(t)]^2}$である<a name="cite_ref-14"></a>[<sup>[14]</sup>](#cite_note-14)。長さの単位はau（天文単位；astronomical unit；1 auは、ほぼ地球と太陽との平均距離に等しい）、時間の単位は年（以後yrと書く）をとることにする。このとき、万有引力定数と太陽質量の積は、

<!-- textlint-enable -->

$$
GM \simeq 4 \pi^2 \ \text{au$^3$/yr$^2$} ,
\tag{21}
$$

となる<a name="cite_ref-15"></a>[<sup>[15]</sup>](#cite_note-15)。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

連立微分方程式<!-- eqref -->(17)-<!-- eqref -->(20)をもとに、オイラー法を用いて、天体の運動をシミュレーションしてみよう。平面上の運動では$t = 0$での初期条件として$x$方向と$y$方向のそれぞれの位置と速度に対応する4つのパラメーターが必要である。ここでは楕円軌道を考えて、「天体の初期位置は太陽からもっとも遠い軌道上の点にとる<a name="cite_ref-16"></a>[<sup>[16]</sup>](#cite_note-16)<a name="cite_ref-17"></a>[<sup>[17]</sup>](#cite_note-17)、移動方向は半時計周りとする」という条件を課す。さらに、天体の初期位置を$x$軸の正の方向とすると（$v_x(0)=0$、$y(0)=0$であるから）残りの自由なパラメーターは2つである。これらを楕円軌道<a name="cite_ref-18"></a>[<sup>[18]</sup>](#cite_note-18)の軌道長半径$a$と軌道離心率$e$で決めることにすると、次のような式が得られる：

<!-- textlint-enable -->

$$
\begin{align}
x(0) &= r_0 , \tag{22} \\
y(0) &= 0 , \tag{23} \\
v_x(0) &= 0, \tag{24} \\
v_y(0) &= r_0 \omega_0 . \tag{25}
\end{align}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

ただし、

<!-- textlint-enable -->

$$
r_0 = a (1 + e) , \qquad
\omega_0 = \frac{\sqrt{GMa(1-e^2)}}{r_0^2} , \tag{26}
$$

である。

(1) 上記の式に従い、天体の運動のシミュレーションをオイラー法で行うプログラムを作成せよ。チェックとして、まず地球の軌道のパラメーター<a name="cite_ref-19"></a>[<sup>[19]</sup>](#cite_note-19)：

$$
a = 1.00000102 \text{ au}, \qquad
e = 0.0167086 ,
\tag{27}
$$

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/sentence-length -->

を用い、$\Delta t = 0.00001 \text{ yr}$として、$t_\text{max} = 3 \text{ yr}$程度まで計算してみよう。$x$-$y$ 平面での軌道をグラフに描くと、ほぼ円軌道となっているだろう<a name="cite_ref-20"></a>[<sup>[20]</sup>](#cite_note-20)<a name="cite_ref-21"></a>[<sup>[21]</sup>](#cite_note-21)。また、$x$座標（もしくは$y$座標でもよいが）を時間$t$の関数として描くと、公転周期が1年となっているはずである。それらを確認したら、提出する際には次のパラメーターで表される架空の天体の運動を少なくとも$t_\text{max} = 3 \text{ yr}$以上まで計算し、$x$-$y$平面での軌道をグラフとして描くようにせよ。

<!-- textlint-enable -->

$$
a = 1.6 \text{ au} , \qquad
e = 0.5 .
\tag{28}
$$

<!-- textlint-disable ja-engineering-paper/use-si-units -->

なお、この架空の天体の公転周期は約2.02 yrであり、近日点距離が0.8 au、遠日点距離が2.4 auとなるような楕円軌道を描く。

<!-- textlint-enable -->

In [ ]:
# 課題解答13.1  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## 発展課題

<!-- textlint-disable jtf-style/1.1.1.本文 -->

余裕があればやってみてください。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-engineering-paper/use-si-units -->

(2) ハレー彗星（公転周期75.32 yr<a name="cite_ref-22"></a>[<sup>[22]</sup>](#cite_note-22)、近日点距離0.586 au、遠日点距離35.082 au）の軌道

<!-- textlint-enable -->

$$
a = 17.834 \text{ au} , \qquad
e = 0.96714 ,
\tag{29}
$$

<!--
理科年表の軌道要素の値の精度が悪くてハレー彗星の周期が半年ほどずれる。。。
-->

<!-- textlint-disable ja-engineering-paper/use-si-units -->

についてシミュレーションしてみよう。演習課題(1)で作ったプログラムにおいて同じステップ幅を使って80 yrほど計算すると、残念ながらオイラー法の誤差によって楕円軌道から少しずれてしまう（元の位置に戻ってこない）。ステップ幅を小さくしたり、ルンゲ＝クッタ法（Runge–Kutta method）のような高次の近似法を用いたりしてもよいが、ここでは以下のことを考えよう。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

いま、保存力のみを考えているので、運動方程式は

<!-- textlint-enable -->

$$
\begin{align}
\frac{d\vec{r}(t)}{dt} &= \vec{v}(t) , \tag{30} \\
m \frac{d\vec{v}(t)}{dt} &= \vec{f}\left(\vec{r}(t)\right) = - \vec{\nabla} U(\vec{r}) , \tag{31}
\end{align}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

のような形で与えられている。$U(\vec{r})$は（スカラー）ポテンシャルである。これを

<!-- textlint-enable -->

$$
\begin{align}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t) \Delta t, \tag{32} \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \frac{1}{m} \vec{f}\left(\vec{r}(t)\right) \Delta t , \tag{33}
\end{align}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

と近似するのがオイラー法であった。式<!-- eqref -->(32)と<!-- eqref -->(33)の写像は力学的エネルギーを保存せず、繰り返し適用することによって力学的エネルギーが変化してしまう。ここで、式<!-- eqref -->(32)と<!-- eqref -->(33)の代わりに

<!-- textlint-enable -->

$$
\begin{align}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t) \Delta t, \tag{34} \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \frac{1}{m} \vec{f}\left(\vec{r}(t+\Delta t)\right) \Delta t , \tag{35}
\end{align}
$$

を使うと、力学的エネルギー保存に対する誤差を改善できることが知られている（式<!-- eqref -->(35)の右辺第二項に注意）。この近似法はシンプレクティック・オイラー法（symplectic Euler method）と呼ばれている<a name="cite_ref-23"></a>[<sup>[23]</sup>](#cite_note-23)。これを使ってみよう。

<!-- textlint-disable ja-technical-writing/sentence-length -->

まず、演習課題(1)のようにオイラー法で地球の軌道を計算するとき、ステップ幅を$\Delta t = 0.001 \text{ yr}$とすると、誤差により軌道が閉じない（1周するとずれる）ことを確認する。次に、プログラムをシンプレクティック・オイラー法を使うように書き換えると、$\Delta t = 0.001 \text{ yr}$のままでも軌道が閉じるはずである<a name="cite_ref-24"></a>[<sup>[24]</sup>](#cite_note-24)。このことが確認できたら、ハレー彗星の軌道計算を少なくとも$t_\text{max} = 150 \text{ yr}$以上行い、$x$-$y$平面での軌道をグラフに描け。

<!-- textlint-enable -->

In [ ]:
# 課題解答13.2  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

(3) 中心力は角運動量保存則を導き、平面内での運動をもたらす。しかし、天体の軌道が閉じた楕円を描くのは中心力が距離の2乗に反比例しているか、もしくは調和振動子の場合のみである。実際、式<!-- eqref -->(16)の代わりに

<!-- textlint-enable -->

$$
m \frac{d^2 \vec{r}}{dt^2} = - \frac{GMm}{|\vec{r}|^2}
\left( 1 + \frac{C_2}{|\vec{r}|^2} \right)
\frac{\vec{r}}{|\vec{r}|} ,
\tag{36}
$$

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period,ja-engineering-paper/use-si-units -->

を用いると<a name="cite_ref-25"></a>[<sup>[25]</sup>](#cite_note-25)、楕円軌道の長軸の向きが回転する（近日点移動）<a name="cite_ref-26"></a>[<sup>[26]</sup>](#cite_note-26)<a name="cite_ref-27"></a>[<sup>[27]</sup>](#cite_note-27)。このことをシンプレクティック・オイラー法を用いてシミュレーションしてみよう。水星（公転周期0.240846 yr、近日点距離0.307499 au、遠日点距離0.466697 au）の軌道パラメーター：

<!-- textlint-enable -->

$$
a = 0.387098 \text{ au} , \qquad
e = 0.205630 ,
\tag{37}
$$

<!-- textlint-disable ja-technical-writing/sentence-length -->

を用い、$C_2 = 0.005 \text{ au}^2$<a name="cite_ref-28"></a>[<sup>[28]</sup>](#cite_note-28)、$\Delta t = 0.0001 \text{ yr}$として<a name="cite_ref-29"></a>[<sup>[29]</sup>](#cite_note-29)、$t_\text{max} = 1 \text{ yr}$までシミュレーションして$x$-$y$平面での軌道をグラフに描け<a name="cite_ref-30"></a>[<sup>[30]</sup>](#cite_note-30)。

<!-- textlint-enable -->

In [ ]:
# 課題解答13.3  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## 脚注

<a name="cite_note-1"></a>1.&nbsp;[^](#cite_ref-1)
ガリレオ・ガリレイの有名な言葉に「宇宙は数学の言葉で書かれている（L'universo è scritto in lingua matematica.）」というものがある。私のように数学の苦手な人には残念なお知らせではあるが、数学を知らないと我々の住んでいるこの宇宙を支配している物理法則を理解できない。

<a name="cite_note-2"></a>2.&nbsp;[^](#cite_ref-2)
$n$個の場合が分かりにくければ、まずは$n=1$、つまりスカラー量の場合を考えるとよい。

<a name="cite_note-3"></a>3.&nbsp;[^](#cite_ref-3)
ここで$\vec{X}$は実ベクトル（$\vec{X}\in\mathbb{R}^n$）、$t$は実数とする。$\vec{X}$を複素ベクトル（$\vec{X} \in \mathbb{C}^n$）としてもよいが、それは実部と虚部を分けて考えると$\vec{X} \in \mathbb{R}^{2n}$と等価である。

<!-- textlint-disable ja-technical-writing/sentence-length -->

<a name="cite_note-4"></a>4.&nbsp;[^](#cite_ref-4)
悪条件がなければ、$\Delta t$を小さくしていけばオイラー法の打切り誤差をいくらでも小さくできる（丸め誤差などはある）。時刻$t=0$から$t=T$までを$N$ステップに分割すると$\Delta t = T/N$であり、オイラー法による誤差の蓄積は全体で（十分に$\Delta t$が小さければ）$N \times \mathcal{O}\left((\Delta t)^2\right)=\mathcal{O}(\Delta t)$と期待される。現実的には$\Delta t$を変えながら（たとえば$\Delta t$を2倍にしたりして）2回以上同じ問題を解いてみて、数値解の軌道のずれがどのくらいになるか調べてみる必要があるだろう。

<!-- textlint-enable -->

<a name="cite_note-5"></a>5.&nbsp;[^](#cite_ref-5)
（後退オイラー法などと区別して）前進オイラー法（forward Euler method）と呼ぶこともある。

<a name="cite_note-6"></a>6.&nbsp;[^](#cite_ref-6)
ばね定数$k$のばねにつながれた質量$m$の質点の運動の場合だと、$\omega = \sqrt{k / m}$である。ただし、静止状態を$x=0$とする。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-7"></a>7.&nbsp;[^](#cite_ref-7)
このプログラムでは簡単のため毎ステップごとにリストへと点を追加しているが、グラフの描画のためであれば、そんなに細かく点を保存しておく必要はない。むしろ点が多すぎるとグラフの描画が遅くなる。たとえば100ステップごとに1回だけ点を追加するとか、もしくは後で

<!-- textlint-enable -->

```python
n = 1000  # グラフで使う点の数
d = max(len(t_points) // n, 1)
t_points = t_points[::d]
x_points = x_points[::d]
vx_points = vx_points[::d]
```
のように、グラフに描画する際、適当にダウンサンプリングしてもよいだろう。

<a name="cite_note-8"></a>8.&nbsp;[^](#cite_ref-8)
大きな$\Delta t$の時の軌道を厳密解と比較してもよいし、ばね振り子の場合の力学的エネルギー$E = \frac{1}{2} m v_x^2 + \frac{1}{2} m \omega^2 x^2$（$m$の値は適当にとってよい）がどのくらいの精度で保存しているかどうかを調べてみてもよい（たとえば、初期状態での値との比$(E / E_0)$を考えると$1$に近い値となるべきである）。

<a name="cite_note-9"></a>9.&nbsp;[^](#cite_ref-9)
ボールに回転があるとマグヌス効果が効いてくるだろう。ここでは単に偶然まったくボールが回転しなかったとか、あるいはボールを投げた人がナックルボールの名手だった、とでも思っておくことにしよう。

<!-- textlint-disable
ja-technical-writing/ja-no-mixed-period
-->

<a name="cite_note-10"></a>10.&nbsp;[^](#cite_ref-10)
典型的なボールの大きさと速さでは、速度に比例する粘性抵抗力は、速度の2乗に比例する慣性抵抗力と比べて無視できる。不安であれば、粘性抵抗力をストークスの法則を用い評価し、

<!-- textlint-enable -->

$$
\vec{F}_\text{viscous} = - 6 \pi a \eta \vec{v},
$$

<!-- textlint-disable
ja-technical-writing/sentence-length
-->

これを慣性抵抗力と比べてみるとよい（$a$はボールの半径、$\eta$は空気の粘性係数で$1.8 \times 10^{-5}\ \text{kg}\text{·}\text{m}^{-1}\text{·}\text{s}^{-1}$）。

<!-- textlint-enable -->

<a name="cite_note-11"></a>11.&nbsp;[^](#cite_ref-11)
ソフトボール（質量約0.2 kg）を投げるとして、半径約0.05 m、空気密度1.2 kg/m$^3$、抗力係数$C_D=0.5$とすると、$b = 0.002$ kg/m程度となる（野球のボールに対する[同様の計算](http://www.physics.usyd.edu.au/~cross/TRAJECTORIES/Trajectories.html)を参考にした）。地表での重力加速度が約9.8 m/s$^2$なのはよく知られている。このシミュレーションで一番適当なのはボールの初速度である（このくらい飛ぶかなぁと思って適当に決めた：20 m/s $=$ 72 km/hなので無理ではないだろう）。

<a name="cite_note-12"></a>12.&nbsp;[^](#cite_ref-12)
この場合、空気抵抗があるので力学的エネルギー$E = \frac{1}{2} m v^2 + m g y$は保存しないが、時間とともに力学的エネルギーが減っていく様子を調べてみても面白いだろう。

<a name="cite_note-13"></a>13.&nbsp;[^](#cite_ref-13)
考えている天体は太陽以外からの重力の影響を受けない、自転や扁平率も考えない、天体の質量は太陽の質量に比べてきわめて小さいので太陽は原点にじっとしている、と仮定して問題を簡単化する。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-14"></a>14.&nbsp;[^](#cite_ref-14)
今回のような逆二乗則に従う中心力による運動では、力学的エネルギー$E = \frac{1}{2} m v^2 - \frac{G M m}{r} = - \frac{mGM}{2a}$と角運動量（の大きさ）$|L_z| = m |x v_y - y v_x| = m \sqrt{GMa(1-e^2)}$、ラプラス＝ルンゲ＝レンツ・ベクトル（の各成分）：

<!-- textlint-enable -->

$$
A_i = m^2 (r_i v^2 - v_i \vec{r} \cdot \vec{v}) - m^2 GM \frac{r_i}{r} , \qquad (i = x, y),
$$

が保存量となる。

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-15"></a>15.&nbsp;[^](#cite_ref-15)
天体の軌道長半径を$a$、公転周期を$T$とすると、次の関係が導ける：

<!-- textlint-enable -->

$$
\frac{a^3}{T^2} = G \frac{M + m}{4 \pi^2} .
$$

これに地球の場合（$a = 1 \text{ au}$、$T = 1 \text{ yr}$、$M \gg m$）を当てはめると式<!-- eqref -->(21)となる。$a^3$と$T^2$の比が定数になるというのは、ケプラーの第3法則を表している。

<a name="cite_note-16"></a>16.&nbsp;[^](#cite_ref-16)
$r = r_\text{max} = a(1 + e)$、$\frac{d^2r}{dt^2}=0$となる点なので計算がしやすい。

<a name="cite_note-17"></a>17.&nbsp;[^](#cite_ref-17)
太陽（つまり座標系の原点）は楕円の中心ではなく、（初期位置から遠いほうの）焦点に位置する。

<a name="cite_note-18"></a>18.&nbsp;[^](#cite_ref-18)
楕円の中心から焦点までの距離は$a e$であり、軌道短半径$b$との関係は$a^2 (1 - e^2) = b^2$である。また、$0 \le e < 1$であり、$e = 0$は真円に対応する。

<a name="cite_note-19"></a>19.&nbsp;[^](#cite_ref-19)
ただし、今回の数値計算の精度を考えると、ここまで正確に軌道パラメーターを入力する必要はない。

<a name="cite_note-20"></a>20.&nbsp;[^](#cite_ref-20)
グラフの縦横比を調整するためには[`Axes`オブジェクト](https://matplotlib.org/stable/api/axes_api.html#the-axes-class)の[`set_aspect`メソッド](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_aspect.html)に`"equal"`を渡して呼び出せばよい。

<a name="cite_note-21"></a>21.&nbsp;[^](#cite_ref-21)
ステップ幅が大きいと、誤差のために円軌道が閉じない（1周したときに位置がずれる）。

<!-- textlint-disable jtf-style/1.1.1.本文 -->

<a name="cite_note-22"></a>22.&nbsp;[^](#cite_ref-22)
次にハレー彗星が太陽に接近するのは2061年の夏であり、見かけの等級は最大で$-0.3$となるそうなので、みなさんご期待ください。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

<a name="cite_note-23"></a>23.&nbsp;[^](#cite_ref-23)
式<!-- eqref -->(34)と<!-- eqref -->(35)の代わりに

<!-- textlint-enable -->

$$
\begin{align*}
\vec{r}(t+\Delta t) &\simeq \vec{r}(t) + \vec{v}(t+\Delta t) \Delta t, \\
\vec{v}(t+\Delta t) &\simeq \vec{v}(t) + \frac{1}{m} \vec{f}\left(\vec{r}(t)\right) \Delta t ,
\end{align*}
$$

としてもよい。シンプレクティック・オイラー法は、ハミルトン力学系に特化したシンプレクティック数値積分法（symplectic integrator）のもっとも簡単な例である。

<a name="cite_note-24"></a>24.&nbsp;[^](#cite_ref-24)
力学的エネルギーの値をグラフにしてみると、多少の変化はするが周期的に元の値に戻るのが分かる（そうなっていない場合はおそらく式を間違えている）。

<a name="cite_note-25"></a>25.&nbsp;[^](#cite_ref-25)
一般相対性理論より式<!-- eqref -->(36)の$C_2 / r^2$の補正項が導出される。たとえば次の文献を参考にせよ。

<!-- textlint-disable ja-technical-writing/sentence-length,ja-technical-writing/max-comma -->

- James D. Wells, *When effective theories predict: the inevitability of Mercury's anomalous perihelion precession*, In: *Effective Theories in Physics: From Planetary Orbits to Elementary Particle Masses* (Springer, 2012), [arXiv:1106.1568](https://arxiv.org/abs/1106.1568).

<!-- textlint-enable -->

<a name="cite_note-26"></a>26.&nbsp;[^](#cite_ref-26)
この場合、力学的エネルギー$E = \frac{1}{2} m v^2 - \frac{GMm}{r} \left(1 + \frac{C_2}{3r^2}\right)$と角運動量（の大きさ）$|L_z| = m |x v_y - y v_x|$は保存するが、ラプラス＝ルンゲ＝レンツ・ベクトルは保存しない。

<a name="cite_note-27"></a>27.&nbsp;[^](#cite_ref-27)
太陽系の惑星に働く重力はその大部分が太陽によるものであるが、ほかの惑星からの重力の影響も受けており、完全な楕円軌道を描いているわけではない。実際、海王星は天王星の軌道予測と観測とのずれが基となって発見されている。

<!-- textlint-disable ja-technical-writing/sentence-length -->

水星の近日点移動もその大部分は他の惑星からの重力による摂動などニュートン力学によって説明できる。しかし、水星の近日点移動の観測値が1世紀あたり約574″（秒；1″は1°の1/3600）であるのに対し、ニュートン力学によって説明可能なのはそのうちの約531″であり、残りの約43″のずれは19世紀の天文学者たちを大いに悩ませた。天王星に対する海王星のように、水星の軌道を乱す未知の惑星ヴァルカンが水星より内側の軌道を公転していると考えられたこともあった。この状況は、1915年にアルベルト・アインシュタインが一般相対性理論からこのずれを導出してみせるまで続く。

<!-- textlint-enable -->

<!-- textlint-disable ja-engineering-paper/prh,ja-technical-writing/ja-no-weak-phrase -->

<a name="cite_note-28"></a>28.&nbsp;[^](#cite_ref-28)
軌道をグラフに描いたとき分かりやすいよう、非常に大きな値にとってある（しかし、近日点でも$C_2 / r^2 \approx 0.05 \ll 1$なので「補正項」と言って差し支えないだろう）。プログラムのチェックとしては、一度$C_2 = 0 \text{ au}^2$を代入して計算してみてもよいかもしれない。通常の水星の軌道が描けるはずである。

<!-- textlint-enable -->

<!-- textlint-disable ja-technical-writing/ja-no-mixed-period -->

実際、シュヴァルツシルト計量を用いて計算すると

<!-- textlint-enable -->

$$
C_2 = \frac{a (1-e^2) GM}{c^2} \simeq 3.7 \times 10^{-9} \text{ au$^2$} ,
$$

となり（$c$は光速）、これは1世紀あたり約43″の近日点移動を引き起こす。

<a name="cite_note-29"></a>29.&nbsp;[^](#cite_ref-29)
発展課題(2)のように$\Delta t = 0.001 \text{ yr}$にとると、（目で見える程度に）ちょっとずれる。

<a name="cite_note-30"></a>30.&nbsp;[^](#cite_ref-30)
ちなみに$t_\text{max} = 10 \text{ yr}$くらいまで計算すると、ちょっと綺麗な模様が描ける。それでも提出時には、できれば$t_\text{max} = 1 \text{ yr}$にしておいてください（その方が成否を判断しやすいので）。